# Inteligencia Artificial, Machine Learning 2020-2021 - UIB

Por: Asier Miguel Alemany Ortiz

Repositorio: [https://github.com/Asmial/IAML2021](https://github.com/Asmial/IAML2021)

FIFA es una serie de videojuegos o simuladores de fútbol licenciados, publicados anualmente por Electronic Arts (EA). Durante los últimos 25 años han ido añadiendo características, que entre ellas destaca una mayor importancia en el tema de la simulación de un mánager, sobretodo en la parte de compra-venta de jugadores. Por eso, cada vez se dispone de una base de datos más extensa y detallada de cada uno de los jugadores.

Uno de los elementos más importantes es el valor económico de cada unos de los jugadores. Este valor se supone que está relacionado con las características de cada uno. Por eso, en limpiaremos la base de datos y generaremos un csv en el que junto con la librería scikit-learn, podamos predecir el valor del jugador a partir de sus características.

## Inicialización

Cargamos las librerías necesarias

In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, \
r2_score

import pandas as pd
import numpy as np

import datetime

Cargamos el archivo CSV y le hacemos un análisis visual

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(os.path.join('in', 'fifa.csv'))
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Como podemos ver hay campos que ya de por sí son inútiles porque no aportan nada de información relevante, procedemos a eliminarlos en la siguienete sección.

## Eliminación de columnas innecesarias

Como los URLs son sencillamente archivos de imágen que llevan a una página 404, no nos isrven para analizarlos

In [3]:
df = df.drop(['Photo','Flag','Club Logo'], axis=1)

Como el id y Unnamed: 0 son sencillamente variables para indexar al jugador, las eliminamos tambíen

In [4]:
df = df.iloc[:, 2:]

El nombre y el número de la camiseta son técnicamente irrelevantes excepto para el número 1, que es el portero principal, pero como ya tenemos la posición en la columna Position, vamos a comprobar si hay alguien que tenga el 1 y no sea 'GK' (Goal Keeper).

In [5]:
len(df.loc[(df['Jersey Number'] == 1) & (df['Position'] != 'GK')])

0

Como podemos ver, ambas son variables personales, así que las eliminamos

In [6]:
df = df.drop(['Name','Jersey Number'], axis = 1)

## Estadísticas

Eliminamos las filas de las estadísticas nulas

In [7]:
df = df.dropna(subset= ['Crossing'])

Como podemos ver arriba, el portero De Gea tiene nan en todas las posiciones que no son GK, por eso miraremos si hay mas porteros que tienen ese problema

In [8]:
df.loc[(df['Position'] == 'GK')]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
9,25,Slovenia,90,93,Atlético Madrid,€68M,€94K,1331,Right,3.0,3.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 16, 2014",NaN,2021,6'2,192lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,15.0,29.0,13.0,12.0,13.0,14.0,26.0,16.0,43.0,60.0,67.0,86.0,49.0,22.0,76.0,41.0,78.0,12.0,34.0,19.0,11.0,70.0,11.0,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,€144.5M
18,26,Germany,89,92,FC Barcelona,€58M,€240K,1328,Right,3.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 1, 2014",NaN,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M
19,26,Belgium,89,90,Real Madrid,€53.5M,€240K,1311,Left,4.0,2.0,1.0,Medium/ Medium,Courtois,Yes,GK,"Aug 9, 2018",NaN,2024,6'6,212lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,13.0,33.0,12.0,13.0,19.0,20.0,35.0,23.0,46.0,52.0,61.0,84.0,45.0,36.0,68.0,38.0,70.0,17.0,23.0,15.0,13.0,44.0,27.0,66.0,20.0,18.0,16.0,85.0,91.0,72.0,86.0,88.0,€113.7M
22,32,Germany,89,89,FC Bayern München,€38M,€130K,1473,Right,5.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 1, 2011",NaN,2021,6'4,203lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18178,18,Sweden,48,65,Dalkurd FF,€50K,€1K,738,Right,1.0,2.0,1.0,Medium/ Medium,Lean,No,GK,"Apr 4, 2018",NaN,2018,6'0,176lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,7.0,11.0,22.0,6.0,8.0,10.0,8.0,20.0,12.0,24.0,28.0,33.0,40.0,29.0,15.0,48.0,20.0,38.0,6.0,20.0,10.0,7.0,18.0,12.0,28.0,16.0,11.0,10.0,47.0,46.0,50.0,45.0,51.0,€94K
18180,22,Scotland,48,58,St. Johnstone FC,€40K,€1K,987,Right,1.0,2.0,1.0,Medium/ Medium,Lean,No,GK,"Jul 1, 2013",NaN,2019,6'1,172lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,15.0,15.0,26.0,14.0,13.0,12.0,19.0,19.0,25.0,40.0,42.0,45.0,47.0,52.0,25.0,55.0,33.0,58.0,20.0,20.0,29.0,13.0,20.0,29.0,28.0,12.0,15.0,16.0,45.0,49.0,50.0,50.0,45.0,€78K
18183,44,England,48,48,Cambridge United,€0,€1K,774,Right,1.0,2.0,1.0,Medium/ Medium,Normal,No,GK,"Aug 17, 2018",N

Como podemos ver, se cumple, así que pasaremos esos datos a '0+0'

In [9]:
posattr = ['LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM', \
           'LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB', \
           'LB','LCB','CB','RCB','RB']
for i in posattr:
    df[i] = df[i].fillna('0+0')

Comprobamos si el numero de la izquierda del '+' es dependiente de el de la derecha

In [10]:
wplus = 0
woplus = 0
for i in posattr:
    wplus += len(df['ST'].str[0:-2].unique())
    woplus += len(df['ST'].unique())
print(wplus)
print(woplus)

1560
2444


No parece que lo sean, por eso mismo los trataremos distinto.

Suponemos que lo de la derecha del + es el potencial máximo, por eso, haremos la media entre la base y la base más el máximo, ya que un jugador con 50+3 es peor que uno con 52+1 debido a que la base inicial es superior.

In [11]:
dfc = df.copy()

def plusclean(x):
    y = x.split('+')
    return int(y[0]) + int(y[1])/2

for i in posattr:
    dfc[i] = dfc[i].apply(lambda x: plusclean(x))

df = dfc

In [12]:
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,89.0,89.0,89.0,93.0,94.0,94.0,94.0,93.0,94.0,94.0,94.0,92.0,85.0,85.0,85.0,92.0,65.0,62.0,62.0,62.0,65.0,60.0,48.0,48.0,48.0,60.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,92.5,92.5,92.5,90.5,91.5,91.5,91.5,90.5,89.5,89.5,89.5,89.5,82.5,82.5,82.5,89.5,66.5,62.5,62.5,62.5,66.5,62.5,54.5,54.5,54.5,62.5,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,85.5,85.5,85.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,89.5,82.5,82.5,82.5,89.5,66.5,61.5,61.5,61.5,66.5,61.5,48.5,48.5,48.5,61.5,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,83.5,83.5,83.5,88.5,88.5,88.5,88.5,88.5,89.5,89.5,89.5,89.5,88.5,88.5,88.5,89.5,78.5,78.5,78.5,78.5,78.5,74.5,67.5,67.5,67.5,74.5,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,€60K,€1K,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,"May 3, 2017",NaN,2019,5'9,134lbs,43.0,43.0,43.0,45.0,45.0,45.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,46.0,46.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,19,Sweden,47,63,Trelleborgs FF,€60K,€1K,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,46.0,46.0,46.0,40.0,43.0,43.0,43.0,40.0,41.0,41.0,41.0,39.0,36.0,36.0,36.0,39.0,31.0,32.0,32.0,32.0,31.0,30.0,33.0,33.0,33.0,30.0,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,

## Precios

En la tabla hay varios valores numéricos indicando el precio, vamos a analizar sus prefijos y sufijos

In [13]:
pricescols = ['Value', 'Wage', 'Release Clause']
df = df.dropna(subset= pricescols)
for i in pricescols:
    for j in [0,-1]:
        print(pd.Series(df[i].str.strip().str[j]).unique())

['€']
['M' 'K']
['€']
['K']
['€']
['M' 'K']


Posibles monedas: €

Posibles terminadores: K, M

Los convertimos a floats según los prefijos y sufijos:

In [14]:
newnames = ['value', 'wage', 'rclause']
dfc = df.copy()
for count, col in enumerate(pricescols):
    dfc[newnames[count]] = df[col].apply(lambda x:  \
              float(x[1:-1]) * 1000 if x[-1] == 'K' \
                      else float(x[1:-1]) * 1000000 )
df = dfc
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,value,wage,rclause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,89.0,89.0,89.0,93.0,94.0,94.0,94.0,93.0,94.0,94.0,94.0,92.0,85.0,85.0,85.0,92.0,65.0,62.0,62.0,62.0,65.0,60.0,48.0,48.0,48.0,60.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M,110500000.0,565000.0,226500000.0
1,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,92.5,92.5,92.5,90.5,91.5,91.5,91.5,90.5,89.5,89.5,89.5,89.5,82.5,82.5,82.5,89.5,66.5,62.5,62.5,62.5,66.5,62.5,54.5,54.5,54.5,62.5,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M,77000000.0,405000.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,85.5,85.5,85.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,89.5,82.5,82.5,82.5,89.5,66.5,61.5,61.5,61.5,66.5,61.5,48.5,48.5,48.5,61.5,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M,118500000.0,290000.0,228100000.0
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M,72000000.0,260000.0,138600000.0
4,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,83.5,83.5,83.5,88.5,88.5,88.5,88.5,88.5,89.5,89.5,89.5,89.5,88.5,88.5,88.5,89.5,78.5,78.5,78.5,78.5,78.5,74.5,67.5,67.5,67.5,74.5,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M,102000000.0,355000.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,€60K,€1K,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,"May 3, 2017",NaN,2019,5'9,134lbs,43.0,43.0,43.0,45.0,45.0,45.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,46.0,46.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K,60000.0,1000.0,143000.0
18203,19,Sweden,47,63,Trelleborgs FF,€60K,€1K,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,46.0,46.0,46.0,40.0,43.0,43.0,43.0,40.0,41

Eliminamos las columnas anteriores de los precios en forma de string

In [15]:
df = df.drop(pricescols, axis=1)

In [16]:
df

,Age,Nationality,Overall,Potential,Club,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause
0,31,Argentina,94,94,FC Barcelona,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,89.0,89.0,89.0,93.0,94.0,94.0,94.0,93.0,94.0,94.0,94.0,92.0,85.0,85.0,85.0,92.0,65.0,62.0,62.0,62.0,65.0,60.0,48.0,48.0,48.0,60.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,110500000.0,565000.0,226500000.0
1,33,Portugal,94,94,Juventus,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,92.5,92.5,92.5,90.5,91.5,91.5,91.5,90.5,89.5,89.5,89.5,89.5,82.5,82.5,82.5,89.5,66.5,62.5,62.5,62.5,66.5,62.5,54.5,54.5,54.5,62.5,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,77000000.0,405000.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,85.5,85.5,85.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,89.5,82.5,82.5,82.5,89.5,66.5,61.5,61.5,61.5,66.5,61.5,48.5,48.5,48.5,61.5,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,118500000.0,290000.0,228100000.0
3,27,Spain,91,93,Manchester United,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,72000000.0,260000.0,138600000.0
4,27,Belgium,91,92,Manchester City,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,83.5,83.5,83.5,88.5,88.5,88.5,88.5,88.5,89.5,89.5,89.5,89.5,88.5,88.5,88.5,89.5,78.5,78.5,78.5,78.5,78.5,74.5,67.5,67.5,67.5,74.5,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,102000000.0,355000.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,"May 3, 2017",NaN,2019,5'9,134lbs,43.0,43.0,43.0,45.0,45.0,45.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,46.0,46.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,60000.0,1000.0,143000.0
18203,19,Sweden,47,63,Trelleborgs FF,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,46.0,46.0,46.0,40.0,43.0,43.0,43.0,40.0,41.0,41.0,41.0,39.0,36.0,36.0,36.0,39.0,31.0,32.0,32.0,32.0,31.0,30.0,33.0,33.0,33.0,30.0,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,4

## Atributos del personaje

Estos atributos són meramente atributos que son como se refleja el personaje en el juego y deberían de tener poca influencia en el juego, igulamente, quizás estas proporciones hagan que el jugador esté más cotizado, por eso, los trataremos. También trataremos cosas como si el personaje tiene un modelo único en el juego, tanto de cara como de foto, ya que la fama del personaje puede tener una influencia directa sobre el precio.

El peso está en libras, quitamos el sufijo y lo convertimos a int

In [17]:
dfc = df.copy()
dfc['weight'] = dfc['Weight'].apply(lambda x: int(x[0:-3]))
dfc = dfc.drop(['Weight'], axis=1)
df = dfc

La altura está en el formato inglés, como no es un formato en base 10, lo convertimos a centímetros

In [18]:
def tocm(x):
    y = x.split("'")
    return int(y[0]) * 30.48 + int(y[1]) * 2.54

dfc = df.copy()
dfc['height'] = dfc['Height'].apply(lambda x: tocm(x))
dfc = dfc.drop(['Height'], axis=1)
df = dfc

El pie prefierido del jugador es importante para nuestra tabla, como solo puede ser o derecho o izquierdo, ponemos un booleano que si es 1, prefiere usar el pie derecho.

In [19]:
df = df.dropna(subset = ['Preferred Foot'])

footsplit = df.pop('Preferred Foot')
footsplit.name = 'rfoot'
footsplit.loc[footsplit == 'Right'] = 1
footsplit.loc[footsplit == 'Left'] = 0

df = pd.concat([df.reset_index(drop=True),footsplit.reset_index(drop=True)], axis=1, sort=False)

Como se ha dicho antes, que tenga una cara modelada es importante, así que haremos lo mismo que con el pie derecho

In [20]:
df = df.dropna(subset = ['Real Face'])

rf = df.pop('Real Face')
rf.name = 'rf'
rf.loc[rf == 'Yes'] = 1
rf.loc[rf == 'No'] = 0

df = pd.concat([df.reset_index(drop=True),rf.reset_index(drop=True)], axis=1, sort=False)

Comprobamos los tipos de cuerpos:

In [21]:
df['Body Type'].value_counts()

Normal                 9731
Lean                   5856
Stocky                 1049
Akinfenwa                 1
C. Ronaldo                1
Messi                     1
Courtois                  1
Shaqiri                   1
PLAYER_BODY_TYPE_25       1
Neymar                    1
Name: Body Type, dtype: int64

Como podemos ver hay cuerpos que su atributo es único, uno de ellos llama la atencion porque no es ningún nombre, lo buscamos a ver quién es:

In [22]:
df.loc[df['Body Type'] == 'PLAYER_BODY_TYPE_25']

,Age,Nationality,Overall,Potential,Club,Special,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Joined,Loaned From,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf
26,26,Egypt,88,89,Liverpool,2146,3.0,3.0,4.0,High/ Medium,PLAYER_BODY_TYPE_25,RM,"Jul 1, 2017",NaN,2023,84.5,84.5,84.5,88.5,87.5,87.5,87.5,88.5,87.5,87.5,87.5,87.5,81.5,81.5,81.5,87.5,71.5,67.5,67.5,67.5,71.5,67.5,58.5,58.5,58.5,67.5,78.0,90.0,59.0,82.0,73.0,89.0,83.0,60.0,72.0,88.0,94.0,91.0,91.0,91.0,88.0,77.0,68.0,84.0,70.0,83.0,63.0,55.0,90.0,82.0,61.0,91.0,38.0,43.0,41.0,14.0,14.0,9.0,11.0,14.0,69500000.0,255000.0,137300000.0,157,175.26,0,1


Mohammed Salah fue uno de los nominados a balón de oro de 2019, eso significa que seguramente su cuerpo si que sea único por su fama. Por eso, a todos los jugadores con un cuerpo único les pondremos que su cuerpo es 'unique':

In [23]:
unique_bodies = ['Akinfenwa','PLAYER_BODY_TYPE_25','Messi','Neymar', \
                 'C. Ronaldo','Shaqiri','Courtois']
btype = df['Body Type'].copy()
for i in unique_bodies:
    btype.loc[btype == i ] = 'unique'
df = df.drop(['Body Type'], axis = 1)
df = pd.concat([df.reset_index(drop=True), btype.reset_index(drop=True)], \
               axis=1, sort=False)

Analizamos el work rate:

In [24]:
df['Work Rate'].unique()

array(['Medium/ Medium', 'High/ Low', 'High/ Medium', 'High/ High',
       'Medium/ High', 'Medium/ Low', 'Low/ High', 'Low/ Medium',
       'Low/ Low'], dtype=object)

El work rate lo dividiremos en alto y bajo (ariba y debajo del '/')

Factorizamos de forma que:
* Low = 0
* Medium = 1
* High = 2

Lo hacemos así porque son categóricos al fin y al cabo (Low < Medium < High)

In [25]:
df = df.dropna(subset = ['Work Rate'])
wrsplit = df.pop('Work Rate').str.split('/ ').array
wrdf = pd.DataFrame(wrsplit, columns = ['wrh','wrl'])

for i in wrdf.columns:
    wrdf[i].loc[wrdf[i] == 'High'] = 2
    wrdf[i].loc[wrdf[i] == 'Medium'] = 1
    wrdf[i].loc[wrdf[i] == 'Low'] = 0

df = pd.concat([df.reset_index(drop=True),wrdf.reset_index(drop=True)], \
               axis=1, sort=False)

In [26]:
df

,Age,Nationality,Overall,Potential,Club,Special,International Reputation,Weak Foot,Skill Moves,Position,Joined,Loaned From,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf,Body Type,wrh,wrl
0,31,Argentina,94,94,FC Barcelona,2202,5.0,4.0,4.0,RF,"Jul 1, 2004",NaN,2021,89.0,89.0,89.0,93.0,94.0,94.0,94.0,93.0,94.0,94.0,94.0,92.0,85.0,85.0,85.0,92.0,65.0,62.0,62.0,62.0,65.0,60.0,48.0,48.0,48.0,60.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,110500000.0,565000.0,226500000.0,159,170.18,0,1,unique,1,1
1,33,Portugal,94,94,Juventus,2228,5.0,4.0,5.0,ST,"Jul 10, 2018",NaN,2022,92.5,92.5,92.5,90.5,91.5,91.5,91.5,90.5,89.5,89.5,89.5,89.5,82.5,82.5,82.5,89.5,66.5,62.5,62.5,62.5,66.5,62.5,54.5,54.5,54.5,62.5,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,77000000.0,405000.0,127100000.0,183,187.96,1,1,unique,2,0
2,26,Brazil,92,93,Paris Saint-Germain,2143,5.0,5.0,5.0,LW,"Aug 3, 2017",NaN,2022,85.5,85.5,85.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,89.5,82.5,82.5,82.5,89.5,66.5,61.5,61.5,61.5,66.5,61.5,48.5,48.5,48.5,61.5,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,118500000.0,290000.0,228100000.0,150,175.26,1,1,unique,2,1
3,27,Spain,91,93,Manchester United,1471,4.0,3.0,1.0,GK,"Jul 1, 2011",NaN,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,72000000.0,260000.0,138600000.0,168,193.04,1,1,Lean,1,1
4,27,Belgium,91,92,Manchester City,2281,4.0,5.0,4.0,RCM,"Aug 30, 2015",NaN,2023,83.5,83.5,83.5,88.5,88.5,88.5,88.5,88.5,89.5,89.5,89.5,89.5,88.5,88.5,88.5,89.5,78.5,78.5,78.5,78.5,78.5,74.5,67.5,67.5,67.5,74.5,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,102000000.0,355000.0,196400000.0,154,180.34,1,1,Normal,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16638,19,England,47,65,Crewe Alexandra,1307,1.0,2.0,2.0,CM,"May 3, 2017",NaN,2019,43.0,43.0,43.0,45.0,45.0,45.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,46.0,46.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,60000.0,1000.0,143000.0,134,175.26,1,0,Lean,1,1
16639,19,Sweden,47,63,Trelleborgs FF,1098,1.0,2.0,2.0,ST,"Mar 19, 2018",NaN,2020,46.0,46.0,46.0,40.0,43.0,43.0,43.0,40.0,41.0,41.0,41.0,39.0,36.0,36.0,36.0,39.0,31.0,32.0,32.0,32.0,31.0,30.0,33.0,33.0,33.0,30.0,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,60000.0,1000.0,113000.0,170,190.50,1,0,Normal,1,1
16640,16

## Atributos en el equipo

Calculamos el número de días que lleva en el equipo:

In [27]:
dfc = df.copy()
dfc['joined'] = dfc['Joined'].apply(lambda x: \
                              datetime.datetime.strptime(x, '%b %d, %Y'))
dfc = dfc.drop(['Joined'], axis=1)
dfc['joined'] = dfc['joined'].apply(lambda x: \
                             (datetime.datetime.today() - x).days)
df = dfc

Analizamos si el '+' es dependiente de la base

In [28]:
df['Loaned From'].unique()

array([nan], dtype=object)

Vacío, se puede eliminar

In [29]:
df = df.drop(['Loaned From'], axis= 1)

El tipo de contrato válido esté en un tipo incorrecto

In [30]:
df['Contract Valid Until'].dtypes

dtype('O')

Lo pasamos a numérico

In [31]:
df['Contract Valid Until'] = pd.to_numeric(df['Contract Valid Until'], \
                                           errors='coerce')

In [32]:
df['Contract Valid Until'].dtypes

dtype('int64')

In [33]:
df

,Age,Nationality,Overall,Potential,Club,Special,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf,Body Type,wrh,wrl,joined
0,31,Argentina,94,94,FC Barcelona,2202,5.0,4.0,4.0,RF,2021,89.0,89.0,89.0,93.0,94.0,94.0,94.0,93.0,94.0,94.0,94.0,92.0,85.0,85.0,85.0,92.0,65.0,62.0,62.0,62.0,65.0,60.0,48.0,48.0,48.0,60.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,110500000.0,565000.0,226500000.0,159,170.18,0,1,unique,1,1,6053
1,33,Portugal,94,94,Juventus,2228,5.0,4.0,5.0,ST,2022,92.5,92.5,92.5,90.5,91.5,91.5,91.5,90.5,89.5,89.5,89.5,89.5,82.5,82.5,82.5,89.5,66.5,62.5,62.5,62.5,66.5,62.5,54.5,54.5,54.5,62.5,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,77000000.0,405000.0,127100000.0,183,187.96,1,1,unique,2,0,931
2,26,Brazil,92,93,Paris Saint-Germain,2143,5.0,5.0,5.0,LW,2022,85.5,85.5,85.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,89.5,82.5,82.5,82.5,89.5,66.5,61.5,61.5,61.5,66.5,61.5,48.5,48.5,48.5,61.5,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,118500000.0,290000.0,228100000.0,150,175.26,1,1,unique,2,1,1272
3,27,Spain,91,93,Manchester United,1471,4.0,3.0,1.0,GK,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,72000000.0,260000.0,138600000.0,168,193.04,1,1,Lean,1,1,3497
4,27,Belgium,91,92,Manchester City,2281,4.0,5.0,4.0,RCM,2023,83.5,83.5,83.5,88.5,88.5,88.5,88.5,88.5,89.5,89.5,89.5,89.5,88.5,88.5,88.5,89.5,78.5,78.5,78.5,78.5,78.5,74.5,67.5,67.5,67.5,74.5,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,102000000.0,355000.0,196400000.0,154,180.34,1,1,Normal,2,2,1976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16638,19,England,47,65,Crewe Alexandra,1307,1.0,2.0,2.0,CM,2019,43.0,43.0,43.0,45.0,45.0,45.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,46.0,46.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,60000.0,1000.0,143000.0,134,175.26,1,0,Lean,1,1,1364
16639,19,Sweden,47,63,Trelleborgs FF,1098,1.0,2.0,2.0,ST,2020,46.0,46.0,46.0,40.0,43.0,43.0,43.0,40.0,41.0,41.0,41.0,39.0,36.0,36.0,36.0,39.0,31.0,32.0,32.0,32.0,31.0,30.0,33.0,33.0,33.0,30.0,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,60000.0,1000.0,113000.0,170,190.50,1,0,Normal,1,1,1044
16640,16,England,47,67,Cambridge United,1189,1.0,3.0,2.0,ST,2021,46.0,46.0,46.0,46.0,47.0,47.0,47.0,46.0,45.0,45.0,45.0

## Factorización

Factorizamos todas las varibales categóricas:

In [34]:
factors = [['Nationality','nat'],['Position','pos'],['Club','clb'], \
           ['Body Type', 'body']]
for i in factors:
    df.dropna(subset = [i[0]])
    data = df.pop(i[0])
    df = pd.concat([df.reset_index(drop=True), \
            pd.get_dummies(data, prefix=i[1]).reset_index(drop=True)], \
            axis=1, sort=False)

In [35]:
df

,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf,wrh,wrl,joined,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,pos_CAM,pos_CB,pos_CDM,pos_CF,pos_CM,pos_GK,pos_LAM,pos_LB,pos_LCB,pos_LCM,pos_LDM,pos_LF,pos_LM,pos_LS,pos_LW,pos_LWB,pos_RAM,pos_RB,pos_RCB,pos_RCM,pos_RDM,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Ma

## Normalización

Normalizamos todas las variables numéricas excepto value (lo que hay que predecir) y las variables que ya están entre 0 y 1:

In [36]:
for i in df.columns:
    if i != 'value' and df[i].min() != 0 and df[i].max() != 1:
        df[i] = (df[i]-df[i].min())/(df[i].max()-df[i].min())

Por alguna razón, no factoriza los atributos de posición así que lo hacemos a mano

In [37]:
for i in posattr:
    df[i] = (df[i]-df[i].min())/(df[i].max()-df[i].min())

In [38]:
df

,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf,wrh,wrl,joined,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Oman,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,pos_CAM,pos_CB,pos_CDM,pos_CF,pos_CM,pos_GK,pos_LAM,pos_LB,pos_LCB,pos_LCM,pos_LDM,pos_LF,pos_LM,pos_LS,pos_LW,pos_LWB,pos_RAM,pos_RB,pos_RCB,pos_RCM,pos_RDM,pos_RF,pos_RM,pos_RS,pos_RW,pos_RWB,pos_ST,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Ma

## Predicción

Sacamos el valor a predecir:

In [39]:
val = df.pop("value")

Hacemos un split de 1/3 del dataset

In [40]:
X_train, X_test, y_train, y_test = train_test_split(df, val, \
                                                    test_size=0.33, \
                                                    random_state=42)

In [41]:
len(X_train)

11150

Entrenamos el modelo

In [42]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [43]:
preds = reg.predict(X_test)

In [44]:
preds[0]

4788672.0

In [45]:
y_test[0]

110500000.0

Obtenemos el valor de error de R²

In [46]:
r2_score(preds, y_test)

-0.0005879653860292322

Como vemos es bajo, así que nos conformamos con la transformación actual del dataset